In [ ]:
# spaCy 라이브러리: 문장의 토큰화(tokenization), 태깅(tagging) 등의 전처리 기능을 위한 라이브러리
# 영어(Engilsh)와 독일어(Deutsch) 전처리 모듈 설치

In [2]:
!python -m spacy download en # 영어 tokenizer
!python -m spacy download de # 독일어 tokenizer

2023-08-02 11:33:40.837456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 11:33:41.835122: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-02 11:33:43.136665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-02 11:33:43.137120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [1]:
import spacy

# 공식 웹 사이트: https://spacy.io/models/de

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화(tokenization)

In [2]:
# 간단히 토큰화(tokenization) 기능 써보기
# 문장에 존재하는 각 단어(word) 레벨을 토큰으로, 고유한 인덱스를 부여
# 이때, 토큰(token)에는 중요한 의미를 담는 어떤 데이터도 해당될 수 있음
# 우리가 웹 사이트에서 문장을 크롤링한다고 하면, 줄바꿈 태그 <br> 같은 게 같이 수집될 수 있음.
# 이런 태그에 우리가 직접 고유한 인덱스를 달아주는 것도 방법.
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [3]:
# tokenizer를 가볍게 다양하게 사용해보기
tokenized = spacy_en.tokenizer("I am a graduate student. <br> But I do not well at English.")

for i, token in enumerate(tokenized):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .
인덱스 6: <
인덱스 7: br
인덱스 8: >
인덱스 9: But
인덱스 10: I
인덱스 11: do
인덱스 12: not
인덱스 13: well
인덱스 14: at
인덱스 15: English
인덱스 16: .


In [4]:
# 독일어를 영어로 번역하는 task를 해볼 겁니다.
# 독일어를 "input" 즉, 인코더에 들어가는 입력 데이터로 쓸 겁니다.
# seq2seq: 기본 논문에서는 입력 문장을 넣을 때는 토큰 순서를 바꾸어서 넣는다고 했어요.

# 독일어(Deutsch) 문장을 토큰화한 뒤에 순서를 뒤집는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)][::-1]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]


tokenized = tokenize_en("I love a chicken.")
print(tokenized) # 그냥 문장이 들어오면, 단어 단위로 쪼개준다고 생각하면 편함.

['I', 'love', 'a', 'chicken', '.']


In [ ]:
# 최신 torchtext에서는 Field, BucketIterator가 torchtext.legacy.data로 이전됐다고 함.
# 그래서 옛날 버전으로 회귀
# pip => python package 관리 도구 중
!pip install -U torchtext==0.6

# 만약에 런타임 재부팅 하라는 메시지가 나오면, 다시 처음부터 위에서부터 실행

In [6]:
from torchtext.data import Field, BucketIterator


# init_token: <sos> 토큰
# eos_token: <eos> 토큰
# lower는 그냥 대문자를 무시할 것인지 설정(True면 전부 다 소문자로 처리)
# I love you = i love you (대문자가 없어짐) => 토큰 수를 줄일 수 있어요. (|V|를 줄일 수 있음)
SRC = Field(tokenize=tokenize_de, init_token="", eos_token="", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="", eos_token="", lower=True)

In [ ]:
# 총 30,000개의 (독일어, 영어) 쌍으로 구성된 데이터 세트
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz
